In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
#NOTES ON GRADIENT COMPUTATION USING BATCH DATA AS AN INPUT
"""
During training, a batch of data is passed through the model,
which produces predictions for each item in the batch.
The loss is then calculated by comparing these predictions to the
target values for each data pointin the batch, resulting in a vector of individual losses.
This vector is usually averaged or summed to get a single scalar loss for the entire batch.
When .backward() is called on this scalar loss, the gradients are computed with respect to
each model parameter, based on the averaged (or summed) loss for the batch. This gives the
gradient of the batch loss with respect to each parameter.
"""